In [1]:
import scipy.io

import pandas as pd

data = scipy.io.loadmat("/Users/leondeligny/Desktop/PDM/PINN/Raissi/cylinder_nektar_wake.mat")

U_star = data['U_star'] # N x 2 x T
p_star = data['p_star'] # N x T
t_star = data['t'] # T x 1
X_star = data['X_star'] # N x 2

# Averaging U, V and p over the time dimension
U_mean = U_star.mean(axis=2)[:, 0]  # Averaging over time, selecting U component
V_mean = U_star.mean(axis=2)[:, 1]  # Averaging over time, selecting V component
p_mean = p_star.mean(axis=1)  # Average pressure over time

# Create a DataFrame with averaged values
df_train = pd.DataFrame({
    'x': X_star[:, 0],
    'y': X_star[:, 1],
    'u': U_mean,
    'v': V_mean,
    'p': p_mean
})

bc_mask = (df_train['x'] == df_train['x'].min()) | \
       (df_train['x'] == df_train['x'].max()) | \
       (df_train['y'] == df_train['y'].min()) | \
       (df_train['y'] == df_train['y'].max())

df_bc = df_train[bc_mask]

print(df_bc)

             x    y         u         v         p
0     1.000000 -2.0  1.138000 -0.019414 -0.099120
1     1.070707 -2.0  1.137329 -0.013818 -0.098489
2     1.141414 -2.0  1.136308 -0.008371 -0.097512
3     1.212121 -2.0  1.134952 -0.003089 -0.096204
4     1.282828 -2.0  1.133258  0.002001 -0.094559
...        ...  ...       ...       ...       ...
4995  7.717172  2.0  1.015198 -0.006158  0.011689
4996  7.787879  2.0  1.015059 -0.007045  0.011475
4997  7.858586  2.0  1.014865 -0.007948  0.011253
4998  7.929293  2.0  1.014611 -0.008870  0.011071
4999  8.000000  2.0  1.014296 -0.009792  0.010886

[296 rows x 5 columns]


In [2]:
import torch, sys
sys.path.append('/Users/leondeligny/Desktop/PDM/PINN/Raissi')

from PINN_Raissi import PINN_Raissi

mean_variance_dict = {}

# Train the model
model = PINN_Raissi(df_train, bc_mask)
if torch.cuda.device_count() > 1:
    print(f"Let's use {torch.cuda.device_count()} GPUs!")
    model = torch.nn.DataParallel(model)
    
# Check for CUDA availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print(f"Started Training.")
model.train(501)
print(f"Finished Training.")

Layers: [2, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 2]
Started Training.
No checkpoint found at 'path_to_checkpoint.pth', starting from scratch.
It: 0
u_loss: 0.42270371317863464
v_loss: 0.17252925038337708
p_loss: 0.024740705266594887
rans_loss: 0.08319494873285294
Saved checkpoint to 'path_to_checkpoint.pth' at iteration 0
It: 2
It: 4


KeyboardInterrupt: 

In [ ]:
from plot import plot_predictions_vs_test

# Prediction u_pred, v_pred, p_pred, nut_pred
u_pred, v_pred, p_pred = model.predict(df_train)

# Plotting
plot_predictions_vs_test(df_train['x'].astype(float).values.flatten(), df_train['y'].astype(float).values.flatten(), u_pred, df_train['u'], 'u', 'PINN_Raissi_FourierF')
plot_predictions_vs_test(df_train['x'].astype(float).values.flatten(), df_train['y'].astype(float).values.flatten(), v_pred, df_train['v'], 'v', 'PINN_Raissi_FourierF')
plot_predictions_vs_test(df_train['x'].astype(float).values.flatten(), df_train['y'].astype(float).values.flatten(), p_pred, df_train['p'], 'p', 'PINN_Raissi_FourierF')